In [1]:
using LinearAlgebra

In [2]:
function reverse_gauss(A::AbstractMatrix{T}, b::AbstractVector{T}) where T
    x = similar(b)
    N = size(A, 1)
    @inbounds for k in 0:N-1
    x[N-k] = (b[N-k] - sumprod(@view(A[N-k,N-k+1:end]), @view(x[N-k+1:end]))) / A[N-k,N-k]
    end
    return x
end

reverse_gauss (generic function with 1 method)

In [3]:
@inline
function sumprod(A::AbstractVector{T}, B::AbstractVector{T}) where T
    s = T(0)
    @inbounds for i in eachindex(A)
        s = fma(A[i], B[i], s) 
    end
    return s
end

sumprod (generic function with 1 method)

In [4]:
function random_upper_triangular(N::Integer, T::Type)
    A = randn(T, N, N)
    _, A = lu(A)
    return A
end

random_upper_triangular (generic function with 1 method)

In [5]:
Afloat = random_upper_triangular(6, Float64)
bfloat = randn(Float64, 6)
xfloat = reverse_gauss(Afloat, bfloat)

6-element Vector{Float64}:
 -1.2996690265929565
 -2.8419188089433693
 -0.2399923317351978
  0.20750127418148342
 -0.8858205820354792
 -0.17332970280955834

In [6]:
res_float = bfloat - (Afloat*xfloat)

6-element Vector{Float64}:
  1.1102230246251565e-16
 -3.885780586188048e-16
  2.7755575615628914e-17
  0.0
 -1.1102230246251565e-16
  0.0

In [7]:
all(i -> i < 10e-10, res_float)

true

In [8]:
A_bigfloat = random_upper_triangular(6, BigFloat)
b_bigfloat = randn(BigFloat, 6)
x_bigfloat = reverse_gauss(A_bigfloat, b_bigfloat)

6-element Vector{BigFloat}:
  5.067068125223694346444944535615238489704068411075201385650220051694907195481157
  1.530718526528163453056259044781272037107327922461970454388714786746117202095489
 -5.449224228701125189392660436162661291193316039523361709963273512118485059402826
  0.1318806123859138056675337820846489949220566181680058351625650260281271140954846
 -1.050017414163624765640404181553052789238758348426998782818243574137926318946748
  1.850700079740325361022536119690300739490496553634323470474934603256237324105048

In [9]:
res_big_float = b_bigfloat - (A_bigfloat*x_bigfloat)

6-element Vector{BigFloat}:
 -1.20906359771322224755408926079205593995624005102107939390331848582360285244274e-76
 -8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-78
 -2.590850566528333387615905558840119871334800109330884415507111041050577540948728e-77
  4.318084277547222312693175931400199785558000182218140692511851735084295901581214e-78
  8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-78
  0.0

In [10]:
all(i -> i < 10e-10, res_big_float)

true

In [11]:
function transform_to_steps!(A::AbstractMatrix; epsilon = 1e-7)
    @inbounds for k ∈ 1:size(A, 1)
        try
        Δk = leading_element(@view(A[k:end, k]))
        Δk > 1 && swap!(@view(A[k, k:end]), @view(A[k+Δk-1, k:end]))
        for i ∈ k+1:size(A,1)
            t = A[i,k]/A[k,k]
            @. @views A[i,k:end] = A[i,k:end] - t * A[k,k:end]
        end
        catch
            continue
        end
    end
    return A
end

transform_to_steps! (generic function with 1 method)

In [12]:
@inline
function swap!(A,B)
    @inbounds for i in eachindex(A)
        A[i], B[i] = B[i], A[i]
    end
end

swap! (generic function with 1 method)

In [13]:
function leading_element(a::AbstractVector{T}) where T
    absval, k = findmax(abs, a)
    approx_iszero(absval) && throw("Вырожденая матрица")
    return k
end

function leading_element(a::AbstractVector{Rational{T}}) where T
    k = findfirst(x -> !iszero(x), a)
    isnothing(k) && throw("Матрица вырожденная")
    return k
end

leading_element (generic function with 2 methods)

In [14]:
function gauss(A::AbstractMatrix{T}, b::AbstractVector{T}) where T
    Ab = hcat(A, b)
    transform_to_steps!(Ab)

    return reverse_gauss(@view(Ab[1:end, 1:end-1]), @view(Ab[1:end, end]))
end

gauss (generic function with 1 method)

In [15]:
A = randn(1000, 1000)
b = randn(1000)

1000-element Vector{Float64}:
  2.125997620686782
  1.0283063615984263
 -0.05129570028641299
  1.241015773829589
  0.17143481797043103
 -0.5428177823948253
 -1.1506969812416528
  2.0464132835830466
  0.06879121176449374
 -0.4386108319966885
  1.5034403432923373
  0.6106770282684388
 -0.30296083015475606
  ⋮
  0.08844821499096676
  2.1441401890654017
  0.03243411805226193
 -1.0847331879930653
  0.223958336384763
 -0.6369183519307902
  0.9547613224536774
  0.7118780117066037
  0.2770098670254979
 -1.1860932733270848
  1.0614284233071778
 -0.7709747156565975

In [16]:
@time x = gauss(A, b)

  0.270912 seconds (598.45 k allocations: 47.523 MiB, 1.84% gc time, 96.65% compilation time)


1000-element Vector{Float64}:
     -1.7367188745545988e277
     -3.3290152243756276e277
      2.7554485828684103e277
      6.068385101364874e276
     -4.872330084645992e276
      3.117011833957012e276
     -5.556447061118702e275
      6.750242664374158e274
      7.902178986234912e275
      1.3159630374699567e274
     -1.0782774855777491e275
     -4.471949222254569e274
     -2.5268877235863767e274
      ⋮
 -10310.535287480181
  -1317.561333336029
    373.81840983810355
   -909.8005756072462
   -453.7615416467173
    979.8505252476517
    -18.66900711909111
      5.133009076035325
     -0.039734329603128254
      5.678538800301668
     -0.848676607869301
      0.3811166366724383

In [17]:
function matrix_rank(A::AbstractMatrix{T}, eps=10e-7) where T
    step_view = copy(A)
    transform_to_steps!(step_view, epsilon=eps)
    count = 0
    for i in 1:size(A)[1]
        flag = false
        for j in 1:size(A)[2]
            flag = flag || abs(step_view[i, j]) >= eps
        end
        if flag == true count += 1 end
    end
    return count
end

matrix_rank (generic function with 2 methods)

In [18]:
A = randn(1000, 1000)

1000×1000 Matrix{Float64}:
  0.731728   -0.130096   -0.445554   …   2.45924     0.547549    0.648501
  1.18053    -1.21426    -0.370823      -0.579525   -0.403515    1.86749
 -2.2101      0.642292    0.659364      -0.547094   -1.12962     1.61085
  0.353433   -0.72623    -0.0521768     -0.30339    -1.12749     0.0311323
  0.430384   -0.447394    0.225656      -0.0334555  -0.757343   -1.12401
 -1.15851    -0.477451    2.49748    …   0.41145    -0.450054    0.856167
  0.58251    -0.605505   -1.53746        0.903009    1.7015     -0.0590442
  0.0696314  -0.340298   -0.561131      -1.28625    -0.0172841  -0.32084
  0.746297    0.0172404  -0.0403506     -0.535968    0.193989    0.785783
 -0.571214   -0.413365   -0.806718      -1.16603    -0.584538   -1.05939
 -1.41536    -0.782171    0.306138   …   0.747949   -0.0244121   1.04147
 -1.56858     1.42151     0.624301      -0.366388    2.32057     0.315456
  0.280922    1.62036    -1.14499       -1.05541     0.265566    0.711219
  ⋮            

In [19]:
matrix_rank(A) == rank(A)

true

In [20]:
function det_steps(A::AbstractMatrix{T}, eps=10e-9) where T
    if size(A)[1] != size(A)[2] throw("Количество строк и столбцов должно быть одинаковое") end

    step_view = copy(A)
    transform_to_steps!(step_view, epsilon=eps)
    val = one(T)
    for i in 1:size(A)[1]
        val *= step_view[i, i]
    end
    return val
end

det_steps (generic function with 2 methods)

In [21]:
A = randn(3, 3)

3×3 Matrix{Float64}:
 -0.786312    -1.44483   -0.0620529
  0.00459255  -0.145946   1.28674
  0.347815     0.84207   -1.32974

In [22]:
cond(A)

175.1028416604142

In [23]:
det_steps(A) - det(A)

-0.19314685887254696